# SOAI Lezione patica K Nearest Neighbors

Questo è un semplice progetto per mettere in pratica la teorica e il funzionamento dell'algoritmo. Durante questa esercitazione utilizzeremo funzioni e metodi nuovi che semplificheranno i concetti di calcolo.
I punti base di questa esercitazione sono:
1. import delle librerie
2. normalizzazione dei dati
3. applicazione dell'algoritmo
4. valutazione dei risultati 
5. miglioramento dell'algoritmo



### Import delle librerie
Le classiche librerie da importare sono: pandas, numpy, seaborn e matplotlib ma non dimetichiamoci di inserire %matplotlib inline per vedere direttamente sul notebook i grafici che andremo a creare.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Leggiamo il dataset da analizzare
Utilizziamo la funzione .read_scv() di pandas scrivendo il nome del file che in questo caso si trova nella stessa cartella in cui è contenuto il notebook, se fosse stato in un'altra cartella avrei dovuto far precedere al nome del file il percorso assoluto del file

In [30]:
df = pd.read_csv('SOAI-KNN_Data.csv')

**Controllimamo come sono questi dati**

In [28]:
#df.head()

In [29]:
#df.describe()

# Standardizziamo i dati delle variabili

Per rendere omogenei tra le varibili le distanze, rispettando le singole differenze dobbiamo normalizzare i dati standardizzando i valori rispetto ai parametri descrittivi di ciascuna variabile.

Per fare questo utilizziamo una funzione di Scikit learn chiamata StandardScaler, creaiamo un'istanza e gli facciamo analizzare i dati tramite il metodo .fit()

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.drop('TARGET',axis=1))

StandardScaler(copy=True, with_mean=True, with_std=True)

**Ora dobbiamo "trasformare" lo scaler con il metodo .transform() per ottenere un array come quello di partenza ma con i valori standardizzati**

In [6]:
variabili_st = scaler.transform(df.drop('TARGET',axis=1))

Ora non ci resta che ricreare il nuovo dataset con le variabili standardizzate.
**Come si fa?**
Un datataset in pandas può essere creato, senza importare csv, semplicemnte con 2 parametri: l'elenco degli array di variabili  *(nel nostro caso* variabili _st *)* e l'elenco dei nomi di ciascuna variabile *(che nel nostro caso può essere preso da  df.columns)*

In [31]:
df_norm = pd.DataFrame(variabili_st, columns=df.columns[:-1])
#df_norm.head()

In [32]:
#df_norm.describe()

# Dividiamo i dati in due parti (Train Test Split)

**Utilizziamo una funzione di sklern per dividere il dataset in 2 parti una di train e una di test attraverso la funzione  train_test_split**

In [9]:
from sklearn.model_selection import train_test_split

le due righe sottostanti sono identiche la differenza è che nella prima si crea automaticamente un dataset mentre nel secondo si assegna un dataset esistente creato in precedenza.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(variabili_st, df['TARGET'], test_size=0.30, random_state=101)
#X_train

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df_norm, df['TARGET'], test_size=0.30, random_state=101)
#X_train.head()

# Usiamo finalmente il KNN

**Per prima cosa dobbiamo importarlo da sklern.neighbors (KNeighborsClassifier).**

In [16]:
from sklearn.neighbors import KNeighborsClassifier

**Create un modello KNN e istanziamolo con il parametro K --> n_neighbors=1**

In [17]:
MyKnn = KNeighborsClassifier(n_neighbors=1)

**Fit del NOSTRO modello utilizzando il KNN con il seti di dati di training.**

In [18]:
MyKnn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

# Predizioni e Valutazioni del modello 
Ora vediamo come il nostro modello è in grado di prevedere nuovi dati che non ha mai visto prima (test_set)

**Usiamo per questo il metodo predict dell'istanza del nostro modello fornendogli i dati di test_set e assegnamo il risultato ad una variabile.**

In [19]:
pred = MyKnn.predict(X_test)

**Non ci resta che creare la matrice di classificazione (confusion matrix) e il il classification report per vedere come il nostro modello riescea a prevedere questi dati.**

In [20]:
from sklearn.metrics import classification_report,confusion_matrix

In [35]:
print(confusion_matrix(y_test,pred))

In [36]:
print(classification_report(y_test,pred))

# SIAMO SICURI CHE K=1 è la migliore scelta? proviamo altri K! 
Proviamo ad usare il metoto del gomito per trovare un k migliore!

** Costruiamoci una variabile che possa contenere in una lista tutti i valori di errore che si commetterebbe utilizzando vari modelli KNN con differenti valori di K. Facciamo un grafico per visualizzare K ed errore in modo da visualizzare il miglior valore di K al quale riapplicheremo il modello di stima.**

In [24]:
errore = []

# Facciamo un ciclo tante volte
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    errore.append(np.mean(pred_i != y_test))


**Ora creiamo la figura  usando le informazioni del ciclo for appena fatto.**

In [37]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),errore, color='blue', linestyle='--', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Errore vs. Valore di K')
plt.xlabel('K')
plt.ylabel('Errore')

## Riaddestriamo il modell con il nuovo K (esempio k=30)

**Riaddestriamo il modello con il k scelto e riclassifichiamo i dati per valurarne il nuovo adattamento attraverso il  classification report e la matrice di distribuzione.**

In [38]:
# NOW WITH K=30
knn = KNeighborsClassifier(n_neighbors=30)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('Valore di K=30')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))